# idact - prototype demo

## Initial setup

Add `idact` to path:

In [1]:
import sys
import os
import bitmath
import getpass
import contextlib
import fabric

def append_idact_path():
    idact_path = os.path.realpath(os.path.join(os.getcwd(), '../'))
    sys.path.append(idact_path)
append_idact_path()

from idact import *
from idact.detail.auth.set_password import set_password


Hide debug information, setup context manager stack (for testing purposes)

In [2]:
stack = contextlib.ExitStack()
stack.__enter__()
stack.enter_context(fabric.context_managers.hide('everything'))

def exit():
    stack.__exit__(None, None, None)

## Add cluster

In [3]:
cluster = add_cluster(name="test-cluster",
                      user="user-99",
                      host="localhost",
                      port=2222,
                      auth=AuthMethod.ASK)

In [4]:
cluster

Cluster(localhost, 2222, user-99, AuthMethod.ASK)

## Allocate nodes

Cache password (for demo purposes only):

In [5]:
stack.enter_context(set_password(getpass.getpass("Password: ")))

Password: ········


In [6]:
nodes = cluster.allocate_nodes(nodes=4,
                               cores=1,
                               memory_per_node=bitmath.MiB(100),
                               walltime=Walltime(minutes=30),
                               native_args={
                                   '--partition': 'debug'
                               })

In [7]:
nodes

Nodes([Node(NotAllocated),Node(NotAllocated),Node(NotAllocated),Node(NotAllocated)])

In [8]:
nodes.wait()
nodes

Nodes([Node(c6, 2018-06-28 03:00:34.912973+00:00),Node(c7, 2018-06-28 03:00:34.912973+00:00),Node(c8, 2018-06-28 03:00:34.912973+00:00),Node(c9, 2018-06-28 03:00:34.912973+00:00)])

## Run commands

In [9]:
nodes[1].run('whoami')

'user-99'

In [10]:
nodes[1].run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n                24     debug     wrap  user-99  R       0:04      4 c[6-9]'

## Tunnel

In [11]:
tunnel = nodes[0].tunnel(here=9000, there=10000)

In [12]:
tunnel

MultiHopTunnel(9000:10000)

Check tunneling by running `python3 -m http.server 10000` on cluster:

In [13]:
try:
    nodes[0].run('python3 -m http.server 10000', timeout=10)
except TimeoutError:
    pass
finally:
    nodes[0].run('killall python3')

Click the square icon to 
Visit http://localhost:9000,

In [14]:
tunnel.close()

## Deploy notebook

To run Jupyter Notebook on the cluster:

In [15]:
nb = nodes[0].deploy_notebook(local_port=8080)

In [16]:
nb

JupyterDeploymentImpl(8080 -> Node(c6, 2018-06-28 03:00:34.912973+00:00))

In [17]:
nb.local_port

8080

To open the deployed notebook server in a new tab:

In [18]:
nb.open_in_browser()

In this prototype, it's Jupyter Hub: log in using the current user's credentials.

In [19]:
nb.cancel()

## Close

In [20]:
nodes.running()

True

In [21]:
nodes.cancel()

In [22]:
nodes.running()

False

In [23]:
exit()